# ReAct agente desde cero

In [ ]:
# Basado en https://til.simonwillison.net/llms/python-react-pattern

In [2]:
import openai
import re
import os

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")
client = openai.OpenAI()

In [4]:
chat_completion = client.chat.completions.create(
    model = "gpt-4o-mini",
    # model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [5]:
chat_completion.choices[0].message.content

'Hello! How can I assist you today?'

In [7]:
class Agent:
    def __init__(self, system=""):
        # Cada vez que lo llamo, se añade un nuevo mensaje a la conversación.
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        # Almacena el mensaje del usuario y lo envía al modelo.
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        # Almacena la respuesta del modelo.
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # Ejecuta el modelo y devuelve la respuesta.
        completion = client.chat.completions.create(
                        model="gpt-4o", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [8]:
prompt = """
Corres en un ciclo de Pensamiento, Acción, PAUSA, Observación.
Al final del ciclo, das una Respuesta.

Usa el Pensamiento para describir tus pensamientos sobre la pregunta que se te ha hecho.
Usa la Acción para realizar una de las acciones disponibles para ti, luego regresa a PAUSA.
La Observación será el resultado de ejecutar esas acciones.

Tus acciones disponibles son:

calcular:
Ejemplo: calcular: 4 * 7 / 3
Ejecuta un cálculo y devuelve el número, usa la sintaxis de punto flotante si es necesario.

peso_promedio_perro:
Ejemplo: peso_promedio_perro: Collie
Devuelve el peso promedio de un perro cuando se da la raza.

Ejemplo de sesión:

Pregunta: ¿Cuánto pesa un Bulldog?
Pensamiento: Debería buscar el peso del perro usando peso_promedio_perro.
Acción: peso_promedio_perro: Bulldog
PAUSA

Te llamarán nuevamente con esto:

Observación: Un Bulldog pesa 51 libras.

Luego respondes:

Respuesta: Un Bulldog pesa 51 libras.
""".strip()

In [9]:
# Estas son mis dos acciones (agentes)
def calcular(what):
    return eval(what)

def peso_promedio_perro(name):
    if name in "Scottish Terrier": 
        return("El peso promedio de un Scottish Terrier es 20 libras")
    elif name in "Border Collie":
        return("El peso promedio de un Border Collie es 37 libras")
    elif name in "Toy Poodle":
        return("El peso promedio de un Toy Poodle es 7 libras")
    else:
        return("El peso promedio de un perro es 50 libras")



In [10]:
abot = Agent(prompt)

In [11]:
result = abot("Cuánto pesa un toy poodle?")
print(result)

Pensamiento: Debería buscar el peso del perro usando peso_promedio_perro.
Acción: peso_promedio_perro: Toy Poodle
PAUSA


In [12]:
result = peso_promedio_perro("Toy Poodle")
print(result)

El peso promedio de un Toy Poodle es 7 libras


In [13]:
next_prompt = "Observacion: {}".format(result)

In [14]:
abot(next_prompt)

'Respuesta: Un Toy Poodle pesa en promedio 7 libras.'

In [15]:
for idx, message in enumerate(abot.messages):
    print(f"Mensaje {idx + 1}:")
    print(f"  Rol: {message['role']}")
    print(f"  Contenido: {message['content']}")
    print("-" * 40)

Mensaje 1:
  Rol: system
  Contenido: Corres en un ciclo de Pensamiento, Acción, PAUSA, Observación.
Al final del ciclo, das una Respuesta.

Usa el Pensamiento para describir tus pensamientos sobre la pregunta que se te ha hecho.
Usa la Acción para realizar una de las acciones disponibles para ti, luego regresa a PAUSA.
La Observación será el resultado de ejecutar esas acciones.

Tus acciones disponibles son:

calcular:
Ejemplo: calcular: 4 * 7 / 3
Ejecuta un cálculo y devuelve el número, usa la sintaxis de punto flotante si es necesario.

peso_promedio_perro:
Ejemplo: peso_promedio_perro: Collie
Devuelve el peso promedio de un perro cuando se da la raza.

Ejemplo de sesión:

Pregunta: ¿Cuánto pesa un Bulldog?
Pensamiento: Debería buscar el peso del perro usando peso_promedio_perro.
Acción: peso_promedio_perro: Bulldog
PAUSA

Te llamarán nuevamente con esto:

Observación: Un Bulldog pesa 51 libras.

Luego respondes:

Respuesta: Un Bulldog pesa 51 libras.
-------------------------------

In [16]:
abot = Agent(prompt)

In [17]:
question = """Tengo dos perros, un border collie y un scottish terrier. \
Cuanto pesan entre los dos?"""
abot(question)

'Pensamiento: Para responder a esta pregunta, necesito encontrar el peso promedio de un border collie y un scottish terrier, y luego sumar esos dos valores. Usaré la acción peso_promedio_perro para obtener el peso de cada raza.\n\nAcción: peso_promedio_perro: Border Collie\nPAUSA'

In [18]:
next_prompt = "Observacion: {}".format(peso_promedio_perro("Border Collie"))
print(next_prompt)

Observacion: El peso promedio de un Border Collie es 37 libras


In [19]:
abot(next_prompt)

'Pensamiento: Ahora que tengo el peso promedio de un Border Collie, necesito encontrar el peso promedio de un Scottish Terrier para poder sumarlos.\n\nAcción: peso_promedio_perro: Scottish Terrier\nPAUSA'

In [20]:
next_prompt = "Observacion: {}".format(peso_promedio_perro("Scottish Terrier"))
print(next_prompt)

Observacion: El peso promedio de un Scottish Terrier es 20 libras


In [21]:
abot(next_prompt)

'Pensamiento: Ahora que tengo el peso promedio de ambos perros, puedo sumar estos valores para obtener el peso total de los dos perros juntos.\n\nAcción: calcular: 37 + 20\nPAUSA'

In [22]:
next_prompt = "Observacion: {}".format(eval("37 + 20"))
print(next_prompt)

Observacion: 57


In [23]:
abot(next_prompt)

'Respuesta: El peso combinado de un Border Collie y un Scottish Terrier es de 57 libras.'

### Ahora automatizamos en una función

In [24]:
action_re = re.compile('^Acción: (\w+): (.*)$')   # expresión regular para capturar secuencias de texto

acciones_disponibles = {
    "calcular": calcular,
    "peso_promedio_perro": peso_promedio_perro
}

In [ ]:
# Inicia un agente con un prompt base y una pregunta.
# En cada turno, le pasa un prompt al agente y analiza la salida.
# Si la salida indica una acción (detectada por regex), la ejecuta y pasa la observación resultante al siguiente turno del agente.
# Repite hasta agotar max_turns o hasta que no haya más acciones.

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        acciones = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if acciones:
            # There is an action to run
            accion, accion_input = acciones[0].groups()
            if accion not in acciones_disponibles:
                raise Exception("Acción desconocida: {}: {}".format(accion, accion_input))
            print(" -- corriendo {} {}".format(accion, accion_input))
            observacion = acciones_disponibles[accion](accion_input)
            print("Observacion:", observacion)
            next_prompt = "Observacion: {}".format(observacion)
        else:
            return

In [26]:
question = """Tengo dos perros, un border collie y un scottish terrier. \
Cuanto pesan entre los dos?"""
query(question)

Pensamiento: Para responder a esta pregunta, necesito encontrar el peso promedio de un border collie y un scottish terrier, y luego sumar esos dos valores. Primero buscaré el peso del border collie.
Acción: peso_promedio_perro: Border Collie
PAUSA
 -- corriendo peso_promedio_perro Border Collie
Observacion: El peso promedio de un Border Collie es 37 libras
Pensamiento: Ahora que tengo el peso promedio del Border Collie, necesito buscar el peso promedio de un Scottish Terrier.
Acción: peso_promedio_perro: Scottish Terrier
PAUSA
 -- corriendo peso_promedio_perro Scottish Terrier
Observacion: El peso promedio de un Scottish Terrier es 20 libras
Pensamiento: Ahora que tengo los pesos promedio de ambos perros, puedo sumar estos valores para obtener el peso total combinado de los dos perros.
Acción: calcular: 37 + 20
PAUSA
 -- corriendo calcular 37 + 20
Observacion: 57
Respuesta: El peso combinado de un Border Collie y un Scottish Terrier es de 57 libras.
